# Fine-tuning a model with the Trainer API

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


## 🗂️ Data Preparation for Fine-Tuning a Model with Trainer API

**In this section, we:**

- 📥 **Load the MRPC dataset** (sentence pairs with labels for paraphrase detection)
- 🔤 **Load the BERT tokenizer** to convert text into input IDs
- ✂️ **Tokenize the dataset** – prepare sentence pairs for the model
- 🛒 **Set up a data collator** for dynamic padding during batching

In [4]:
# Import the necessary libraries
from datasets import load_dataset                       # For loading datasets
from transformers import AutoTokenizer, DataCollatorWithPadding  # For tokenization and padding

# Load the GLUE MRPC dataset, which contains pairs of sentences and labels indicating if they are paraphrases
raw_datasets = load_dataset("glue", "mrpc")

# Specify the pretrained model checkpoint for BERT (uncased version)
checkpoint = "bert-base-uncased"

# Load the tokenizer corresponding to the pretrained BERT model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Define a function to tokenize pairs of sentences in the dataset, truncating sequences longer than model allows
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Apply the tokenizer function to the entire dataset with batching for speed and efficiency
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Initialize a data collator that dynamically pads inputs in each batch to the longest sequence in that batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

## 🏋️ Training & Fine-Tuning with the Trainer API

**In this section, we:**
- ⚙️ **Define training arguments** (where to save models, how often to evaluate, number of epochs, more)
- 🧠 **Load our classification model** (BERT with a sequence classification head)
- 🤖 **Set up the Trainer** to handle all the training loop and evaluation logic
- 🚀 **Start fine-tuning** the model on our preprocessed dataset

In [3]:
# Import Required Libraries
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification

# Step 1: Define training arguments
training_args = TrainingArguments("test-trainer")

# Step 2: Load model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

# Step 3: Set up Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

# Step 4: Fine-tune modek on our dataset 'MRPC'
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lakshmi-adhikari26 (lakshmi-adhikari26-personalproject) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.552800
1000,0.293800


TrainOutput(global_step=1377, training_loss=0.3427915109095577, metrics={'train_runtime': 272.6267, 'train_samples_per_second': 40.363, 'train_steps_per_second': 5.051, 'total_flos': 405114969714960.0, 'train_loss': 0.3427915109095577, 'epoch': 3.0})

## 📊 Evaluation During Fine-Tuning

**In this section, we:**
- 🔍 Use the `Trainer.predict()` method to get model predictions on the validation set.
- 🎯 Define a `compute_metrics()` function that converts raw model outputs (logits) into class predictions and calculates evaluation metrics like accuracy and F1 score.
- 📈 Incorporate `compute_metrics()` with the `Trainer` to report validation performance automatically during training.

In [5]:
# Import Required Libraries
import numpy as np
import evaluate
from transformers import TrainingArguments,Trainer,AutoModelForSequenceClassification

# Step 1: Define the compute_metrics function
def compute_metrics(eval_preds):
  # Load the standard evaluation metric for MRPC ( from the GLUE benchmark)
  metric = evaluate.load("glue","mrpc")

  # Unpack the predictions and labels from eval_preds tuple
  logits,labels=eval_preds

  # Convert logits to predicted class indices by taking the argmax of each prediction's scores
  predictions = np.argmax(logits,axis=-1)

  # Compute and return the evaluation metrics (accuracy and f1)
  return metric.compute(predictions=predictions,references=labels)

# Step 2: Setup TrainingArguments with evaluation enabled at the of each epoch
training_args = TrainingArguments(
    "test-trainer",
    eval_strategy="epoch",
    fp16=True,  # Use mixed precision
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=3,
  )

# Step 3: Load model (new instance for fresh training with metrics)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

# Step 4: Initialize the Trainer with compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer, # use tokenizer instead of processing_class for latest version
    compute_metrics=compute_metrics # pass our metric function
)

# Step 5: Train the model; metrics will be reported each epoch
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-379657971.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lakshmi-adhikari26 (lakshmi-adhikari26-personalproject) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.456454,0.813725,0.874587
2,No log,0.369663,0.835784,0.879713
3,0.436400,0.421193,0.835784,0.886248


TrainOutput(global_step=690, training_loss=0.3720386615697888, metrics={'train_runtime': 233.7853, 'train_samples_per_second': 47.069, 'train_steps_per_second': 2.951, 'total_flos': 377531475559680.0, 'train_loss': 0.3720386615697888, 'epoch': 3.0})